In [1]:
import os
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import spearmanr #Loocv

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('custom_function/')
import clustering

In [3]:
def cluster_rdm(cluster_results_region,cluster_model='cl_k3'):
    dissim_cluster_result = pd.DataFrame(index = cluster_results_region.index,
                                         columns = cluster_results_region.index)
    for i in dissim_cluster_result.index:
        for j in dissim_cluster_result.columns:
            # cluster result was in the last column
            if cluster_results_region.loc[i,cluster_model] == cluster_results_region.loc[j,cluster_model]:
                dissim_cluster_result.loc[i,j] = 0
            else:
                dissim_cluster_result.loc[i,j] = 1
            dissim_cluster_result = dissim_cluster_result.astype(float)
            
    return(dissim_cluster_result)

In [5]:
relationships = pd.read_csv('input_data/RelDim_labels/159relationships_fix_label.csv',
                           index_col=0)
relationships = relationships['0'].to_list()

# import data

In [7]:
CN_raw_data = pd.read_csv('output_data/cleaning_results/CHN/CHN_dim_rel_scaled.csv', index_col = 0)
HK_raw_data = pd.read_csv('output_data/cleaning_results/HK/HK_dim_rel_scaled.csv',index_col = 0)
UK_raw_data = pd.read_csv('output_data/cleaning_results/UK/UK_dim_rel_scaled.csv',index_col = 0)
ES_raw_data = pd.read_csv('output_data/cleaning_results/Spain/Spain_dim_rel_scaled.csv', index_col = 0)
US_raw_data = pd.read_csv('output_data/cleaning_results/USA/USA_dim_rel_scaled.csv',index_col = 0)
FR_raw_data = pd.read_csv('output_data/cleaning_results/France/France_dim_rel_scaled.csv', index_col = 0)
DE_raw_data = pd.read_csv('output_data/cleaning_results/Germany/Germany_dim_rel_scaled.csv', index_col = 0)
AU_raw_data = pd.read_csv('output_data/cleaning_results/Australia/Australia_dim_rel_scaled.csv', index_col = 0)
CL_raw_data = pd.read_csv('output_data/cleaning_results/Chile/Chile_dim_rel_scaled.csv', index_col = 0)
PT_raw_data = pd.read_csv('output_data/cleaning_results/Portugal/Portugal_dim_rel_scaled.csv', index_col = 0)
MX_raw_data = pd.read_csv('output_data/cleaning_results/Mexico/Mexico_dim_rel_scaled.csv', index_col = 0)
ZA_raw_data = pd.read_csv('output_data/cleaning_results/South_africa/South_africa_dim_rel_scaled.csv', index_col = 0)
BR_raw_data = pd.read_csv('output_data/cleaning_results/Brazil/Brazil_dim_rel_scaled.csv', index_col = 0)
IL_raw_data = pd.read_csv('output_data/cleaning_results/Israel/Israel_dim_rel_scaled.csv', index_col = 0)
RU_raw_data = pd.read_csv('output_data/cleaning_results/Russia/Russia_dim_rel_scaled.csv', index_col = 0)
JP_raw_data = pd.read_csv('output_data/cleaning_results/Japan/Japan_dim_rel_scaled.csv', index_col = 0)
EG_raw_data = pd.read_csv('output_data/cleaning_results/Egypt/Egypt_dim_rel_scaled.csv', index_col = 0)
IN_raw_data = pd.read_csv('output_data/cleaning_results/India/India_dim_rel_scaled.csv', index_col = 0)
QA_raw_data = pd.read_csv('output_data/cleaning_results/Qatar/Qatar_dim_rel_scaled.csv', index_col = 0)

In [8]:
regions = ['USA','UK','South_africa','Australia','Germany','Japan',
            'CHN','HK','France','Chile','Mexico','Spain','Portugal',
            'Brazil','Israel','Russia','Egypt','Qatar','India']
# contain raw output of models
raw_data_dict = {
'CHN' : CN_raw_data, 'HK' : HK_raw_data, 'UK' : UK_raw_data,
'Spain' : ES_raw_data,'USA' : US_raw_data, 'France' : FR_raw_data,
'Germany' : DE_raw_data, 'Australia' : AU_raw_data, 'Chile' : CL_raw_data,
'Portugal' : PT_raw_data, 'Mexico' : MX_raw_data, 'South_africa' : ZA_raw_data,
'Brazil' : BR_raw_data, 'Israel' : IL_raw_data, 'Russia' : RU_raw_data,
'Japan' : JP_raw_data, 'Egypt' : EG_raw_data,'Qatar': QA_raw_data,'India':IN_raw_data}

# clustering results

In [10]:
random_s = 4
for region in regions:
    dim_rel_scaled = raw_data_dict[region]
    dim_rel_scaled.index = relationships
    raw_results = clustering.create_cluster_models(data_type = "df",data = dim_rel_scaled,
                        methods=['KMeans'],ks=[3],keep_orig=True)
    raw_results.to_csv('output_data/categorical_model/raw_dissim/'+\
                      region+'_raw_3k.csv')
    print()
    print('{} 3 KMeans clusters in raw data:'.format(region))
    for n in range(len(raw_results['cl_k3'].unique())):
        each_cluster = raw_results.index[raw_results['cl_k3']==n]
        print("\nCluster "+str(n+1)+" Relationship:{}".format(len(each_cluster)))
        print(*raw_results.index[raw_results['cl_k3']==n], sep = ", ")
        
    print('***************************************************')


USA 3 KMeans clusters in raw data:

Cluster 1 Relationship:68
Employer and Employee, Interviewer and Job applicant, Nurse and Patient, Therapist and Patient, Salesperson and Customer, Between Business Partners, Teacher and Student, Professor and Graduate student, Between teammates in sports, Manager and Staff, Foreman and Laborer, Owner and Manager, Union leader and union member, Manager and Assistant, Chief and Deputy, Supervisor and Subordinate, Boss and Secretary, Entrepreneur and Investor, Intern and Employee, Lawyer and Client, Judge and Lawyer, Drug dealer and Addict, Police officer and Civilian, Consultant and Client, Chef and Customer, Customer and Server, Customer and Businessman, Doctor and Patient, Doctor and Nurse, Patient and Caretaker, Driver and Passenger, Coach and athlete, Team Captain and Team Members, Club Member and Club President, Politician and Supporter, Leader and Follower, Representative and Citizen, Officer and Soldier, Religious leader and Congregant, Guide 

### generate RDM based on cluster results: 0/1 matrix

In [11]:
cluster_raw_dissim_paths = glob.glob('output_data/categorical_model/raw_dissim/*.csv')

# kmeans clustering using raw data
for region_dir in cluster_raw_dissim_paths:
    region_input = pd.read_csv(region_dir, index_col=0)
    region_cluster_rdm = cluster_rdm(region_input,cluster_model='cl_k3')
    
    region_csv = region_dir.split('\\')[1]
    region_cluster_rdm.to_csv('output_data/categorical_model/rdm_results/'+region_csv)